In [ ]:
from latentis.transform.projection import (
    cosine_proj,
    euclidean_proj,
    l1_proj,
    lp_proj,
)
from latentis.methods.bricks_to_bridges import (
    BricksToBridges,
    ConcatAggregation,
    LinearSumAggregation,
    NonLinearSumAggregation,
    WeightedAvgAggregation,
    SelfAttentionLayer,
)
from latentis.space import LatentSpace
from latentis.sample import Uniform

from latentis.nn.decoders import Classifier
from latentis.data.dataset import LatentisDataset
from latentis.data import DATA_DIR

import torch
from latentis.data.text_encoding import HFPooler, cls_pool
from latentis.data.utils import default_collate
from latentis.nn.encoders import TextHFEncoder

from latentis.nexus import space_index

In [ ]:
# import itertools

# from latentis.data.encode import encode_feature
# from latentis.data.processor import TREC

# latentis_dataset = TREC().process(
#     perc=0.05,
# )
# latentis_dataset.save_to_disk()


# for dataset, hf_encoder in itertools.product(["trec"], ["bert-base-cased"]):
#     dataset = LatentisDataset.load_from_disk(DATA_DIR / dataset)

#     encode_feature(
#         dataset=dataset,
#         feature="text",
#         model=TextHFEncoder(hf_encoder),
#         collate_fn=default_collate,
#         encoding_batch_size=256,
#         store_every=10,
#         num_workers=0,
#         save_source_model=False,
#         poolers=[
#             HFPooler(layers=[12], pooling_fn=cls_pool),
#         ],
#         device=torch.device("cpu"),
#     )

In [ ]:
dataset = LatentisDataset.load_from_disk(DATA_DIR / "trec")

test_space = space_index.load_item(split="test", layer=12, **{"model/hf_name": "bert-base-cased"})
test_space.vectors.shape

In [ ]:
test_space_id = space_index.get_item_key(split="test", layer=12, **{"model/hf_name": "bert-base-cased"})

In [ ]:
anchors = test_space.sample(sampler=Uniform(), n=5)
anchors.vectors.shape

In [ ]:
test_space

In [ ]:
test_dataloader = dataset.get_dataloader(
    space_id=test_space_id,
    hf_x_keys=None,
    hf_y_keys=("coarse_label",),
    batch_size=10,
    shuffle=True,
    num_workers=0,
)

it = iter(test_dataloader)
first_batch = next(it)
second_batch = next(it)

In [ ]:
first_batch["encodings_key"]

In [ ]:
bridges = BricksToBridges(
    projection_fns=[cosine_proj, euclidean_proj, l1_proj],
    aggregation_type=ConcatAggregation,
    num_anchors=len(anchors),
)(x=test_space, anchors=anchors, is_pretrained=True)